In [68]:
import os
import timm
import copy
import torch
import open_clip
import numpy as np
import pandas as pd
from torch import nn
from PIL import Image
import pickle as pkl
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

from datasets import build_datasets
from model import PartCEM

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
(dataset_train, dataset_val, dataset_test), attr_indices, class_attrs_df = build_datasets(
    dataset_dir='datasets/CUB',
    attr_subset='cbm',
    use_class_level_attr=True,
    image_size=448
)

In [24]:
len(dataset_train), len(dataset_val), len(dataset_test)

(4795, 1199, 5794)

In [25]:
img_id, img, class_id, attrs = dataset_train[31]
print('img_id:', img_id, img_id.shape, img_id.dtype)
print('class_id:', class_id, class_id.shape, class_id.dtype)
print('attributes:', attrs, attrs.shape, attrs.dtype)

img_id: tensor(8981) torch.Size([]) torch.int64
class_id: tensor(152) torch.Size([]) torch.int64
attributes: tensor([0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.]) torch.Size([112]) torch.float32


In [35]:
img_id, img, class_id, attrs = dataset_train[9]
print('img_id:', img_id, img_id.shape, img_id.dtype)
print('class_id:', class_id, class_id.shape, class_id.dtype)
print('attributes:', attrs, attrs.shape, attrs.dtype)

img_id: tensor(4238) torch.Size([]) torch.int64
class_id: tensor(72) torch.Size([]) torch.int64
attributes: tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]) torch.Size([112]) torch.float32


In [70]:
model = PartCEM()
a, b, c, d =model(torch.rand(4, 3, 448,448))
a.shape, b.shape,c.shape,d

(torch.Size([4, 113, 28, 28]),
 torch.Size([4, 112]),
 torch.Size([4, 200]),
 None)

In [13]:
loss = nn.CrossEntropyLoss()
a = torch.tensor([
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0]
]).to(torch.float32)
b = torch.tensor([
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0]
]).to(torch.float32)
F.binary_cross_entropy(a, b)

tensor(0.)

In [58]:
torch.arange(0,9).view(1,3,3)

tensor([[[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]])

In [14]:
resnet = timm.create_model('resnet50', pretrained=True)

In [17]:
x = resnet.forward_features(torch.rand(1,3,224,224))
x.shape

torch.Size([1, 2048, 7, 7])

In [18]:
resnet.global_pool(x).shape

torch.Size([1, 2048])

In [19]:
resnet.global_pool(torch.randn(1, 2048, 28, 28)).shape

torch.Size([1, 2048])

In [20]:
resnet.global_pool

SelectAdaptivePool2d(pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))

In [42]:
x = torch.rand(1,3,224,224)

In [45]:
torch.equal(x.norm(dim=1,p=2), torch.linalg.vector_norm(x, ord=2, dim=1))

True

In [50]:
fc = nn.Linear(28*28, 1)
x = torch.randn(4, 112, 28*28).view(4, -1, 28*28)
fc(x).squeeze(-1).shape

torch.Size([4, 112])